Derive spec2vec embeddings of MS/MS spectra

In [1]:
import os
import sys
import gensim
import numpy as np

ROOT = os.path.dirname(os.getcwd())
#path_data = os.path.join(ROOT, 'data')
path_data = 'C:\\Users\\Gosia\\Desktop\\'
sys.path.insert(0, ROOT)

Import dataset to create embeddings from, here: pre-processed dataset "Unique InchiKeys"

In [2]:
import passatuto_parser as pp
sys.path.append('C:\\Users\\Gosia\\Desktop\\FDR-Metabolomics\\src\\passatuto_parser.py')


pre_spectrums_query = pp.PassatutoParser(r'C:\\Users\\Gosia\\Desktop\\MassbankOrbi').parse_folder()
pre_spectrums_lib = pp.PassatutoParser('C:\\Users\\Gosia\\Desktop\\Gnps_Noise_Filtered').parse_folder()

# Using MatchMS to create spectra for both
from matchms.importing.load_from_json import as_spectrum
spectrums_query = []
for i, s in enumerate( pre_spectrums_query ):
    spectrums_query.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)
        
spectrums_lib = []
for i, s in enumerate( pre_spectrums_lib ):
    spectrums_lib.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)
        

processed 100 files
processed 200 files
processed 300 files
processed 400 files
Finished parsing of 457 spectra 
processed 100 files
processed 200 files
processed 300 files
processed 400 files
processed 500 files
processed 600 files
processed 700 files
processed 800 files
processed 900 files
processed 1000 files
processed 1100 files
processed 1200 files
processed 1300 files
processed 1400 files
processed 1500 files
processed 1600 files
processed 1700 files
processed 1800 files
processed 1900 files
processed 2000 files
processed 2100 files
processed 2200 files
processed 2300 files
processed 2400 files
processed 2500 files
processed 2600 files
processed 2700 files
processed 2800 files
processed 2900 files
processed 3000 files
processed 3100 files
processed 3200 files
processed 3300 files
processed 3400 files
processed 3500 files
processed 3600 files
processed 3700 files
processed 3800 files
processed 3900 files
processed 4000 files
Finished parsing of 4095 spectra 
processed 100
processe

In [3]:
from matchms.importing import load_from_json
spectrums_lib = []
path_lcms = 'C:\\Users\\Gosia\\Desktop\\gnps_from_simon'
for s in os.listdir(path_lcms):
    spectrums_lib += load_from_json(os.path.join(path_lcms,s))
        

In [4]:
from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses
def post_process_s2v(s):
    
    s = normalize_intensities(s)
    s = select_by_mz(s, mz_from=0, mz_to=1000)
    s = require_minimum_number_of_peaks(s, n_required=10)
    s = reduce_to_number_of_peaks(s, n_required=10, ratio_desired=0.5)
    if s is None:
        return None
    s_remove_low_peaks = select_by_relative_intensity(s, intensity_from=0.001)
    if len(s_remove_low_peaks.peaks) >= 10:
        s = s_remove_low_peaks
        
    s = add_losses(s, loss_mz_from=5.0, loss_mz_to=200.0)
    return s

In [8]:
# apply post processing steps to the data

spectrums_lib = [post_process_s2v(s) for s in spectrums_lib]

# omit spectrums that didn't qualify for analysis

spectrums_lib = [s for s in spectrums_lib if s is not None]


In [ ]:
from matchms.filtering import normalize_intensities
# Spec2Vec trained model requires normalizing
spectrums_query = [normalize_intensities(s) for s in spectrums_query]
spectrums_lib = [normalize_intensities(s) for s in spectrums_lib]


Load pretrained spec2vec model

In [86]:
path_models = os.path.join(path_data, "trained_models")
model_file = os.path.join(path_models, "spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model")

# Load pretrained model
model = gensim.models.Word2Vec.load(model_file)

Create spectrum "documents"

In [10]:
from spec2vec import Spec2Vec
from spec2vec import SpectrumDocument

#documents_query = [SpectrumDocument(s, n_decimals=2) for s in spectrums_query]
documents_lib = [SpectrumDocument(s, n_decimals=2) for s in spectrums_lib]
#print(documents_query)


In [40]:
from spec2vec.model_building import train_new_word2vec_model
path_models = os.path.join(path_data, "trained_models")

model_file = os.path.join(path_models, "spec2vec_librarymatching_size_100.model")

iterations = [1, 3, 5, 10]

#Train model with size 10 and default parameters

model = train_new_word2vec_model(documents_lib, iterations, model_file, size = 100)

The value of size is set from 300 (default) to 100
  Epoch 1 of 10.Change in loss after epoch 1: 1903961.625
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_100_iter_1.model
  Epoch 2 of 10.Change in loss after epoch 2: 1575722.625
  Epoch 3 of 10.Change in loss after epoch 3: 1361473.75
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_100_iter_3.model
  Epoch 4 of 10.Change in loss after epoch 4: 1274022.5
  Epoch 5 of 10.Change in loss after epoch 5: 1234125.5
Saving model with name: C:\Users\Gosia\Desktop\trained_models\spec2vec_librarymatching_size_100_iter_5.model
  Epoch 6 of 10.Change in loss after epoch 6: 1168939.0
  Epoch 7 of 10.Change in loss after epoch 7: 1046637.0
  Epoch 8 of 10.Change in loss after epoch 8: 1020996.0
  Epoch 9 of 10.Change in loss after epoch 9: 1000158.0
  Epoch 10 of 10.Change in loss after epoch 10: 951152.0
Saving model with name: C:\Users\Gosia\Desktop\trained_m


Derive embeddings

In [41]:
from tqdm.notebook import tqdm  # optional, just to get a progress bar
from spec2vec.vector_operations import calc_vector


intensity_weighting_power = 0.5
allowed_missing_percentage = 15 # specify the maximum (weighted) fraction of the spectrum that is allowed to be missing

vector_size = model.vector_size
print(f"Embedding vector size: {vector_size}")

#embeddings_spec2vec_query = np.zeros((len(documents_query), vector_size), dtype="float")
#for i, doc in enumerate(tqdm(documents_query)):
#    embeddings_spec2vec_query[i, 0:vector_size] = calc_vector(model, doc,
#                                                        intensity_weighting_power,
#                                                        allowed_missing_percentage)
embeddings_spec2vec_lib = np.zeros((len(documents_lib), vector_size), dtype="float")
for i, doc in enumerate(tqdm(documents_lib)):
    embeddings_spec2vec_lib[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)

Embedding vector size: 100


In [42]:
#print([np.round(x, 4) for x in embeddings_spec2vec_query[0,:]])
print([np.round(x, 4) for x in embeddings_spec2vec_lib[0,:]])

[-20.2553, 23.3697, 26.3273, -6.411, 59.9514, -8.4595, -18.0223, 12.0185, -33.6996, -29.9557, -19.5061, 25.7586, -13.3538, -34.5308, 53.9283, -19.3537, 7.7151, -6.7035, -37.9293, 5.9646, 1.2801, 58.2668, 38.6722, 55.5007, -6.6895, 18.4156, 20.672, 6.834, 5.4952, -2.1488, 8.936, 23.9831, 15.2567, 23.1338, -73.3666, -26.8928, -1.1262, 15.6832, 9.1843, -20.2924, -1.9166, -29.93, 6.8772, -14.2331, 11.1018, 4.1059, 78.9482, -36.2256, 57.2639, 12.0834, 17.4347, -16.5645, -2.4978, 15.0351, 26.5762, 9.1968, -42.8514, -17.2427, 9.201, -26.8069, -11.5117, -0.2594, 1.751, -8.7163, 0.3586, -20.6555, 52.6081, 39.2642, -0.0907, -39.3073, -21.5782, 30.5351, -15.3938, 17.7743, -1.7567, -49.2557, -20.3511, 29.0435, 1.9563, 61.6495, 25.7956, -1.5794, 57.6926, -21.9423, 8.0085, -6.456, -71.3062, 55.3588, -51.286, 11.3547, 26.7582, -57.767, 42.2818, 90.1023, 65.2442, 25.6384, 63.6896, -67.7287, 36.8816, -26.8009]


# Creating knockoffs


In [43]:
import gensim
from sklearn.mixture import GaussianMixture as GMM
from sklearn.datasets import make_spd_matrix
from spec2vec import calc_vector

In [44]:
#Fitting Gaussian mixture model with 3 components, full covariance structure

gmm = GMM(n_components=25, covariance_type="full")
model = gmm.fit(np.array(embeddings_spec2vec_lib))


In [45]:
#Finding Dk matrix in sampling process

def find_Dk(covariance_matrix, embedding_dimension):
    
    eigs = np.linalg.eig(covariance_matrix)[0]
    min_eig = min(eigs)
    s = min(2*min_eig, 1)
    Dk = np.diag([s]*embedding_dimension)
    return Dk
               
def is_pos_semi_def(A, epsilon = 1e-10):    
    eigs = np.linalg.eig(A)[0]
    min_eig = min(eigs)
    return min_eig >= -epsilon

In [46]:
def create_knockoffs(model,vectors):
    embedding_dimension = len(vectors[0])
    covariances = model.covariances_
    means = model.means_
    Dks = []
    for cov in covariances:
        Dk = find_Dk( cov, embedding_dimension )
        if not is_pos_semi_def( 2*cov-Dk ):
            print('aAAAAAAAAAAAAAAAAaaa')
            return
        Dks.append(Dk)

    knock_means_comps_1 = []
    knock_means_comps_2 = []
    knock_covs = []
    Id = np.diag([1]*embedding_dimension)
    for cov,mean,Dk in zip(covariances,means,Dks):
        knock_cov = 2*Dk - Dk@(cov@Dk)
        knock_mean_comp_1 = Dk@(cov@mean)
        knock_mean_comp_2 = Id - Dk@cov
        knock_means_comps_1.append(knock_mean_comp_1)
        knock_means_comps_2.append(knock_mean_comp_2)
        knock_covs.append(knock_cov)
        
    knockoffs = []
    bad_is = []
    components = np.arange(len(model.weights_))
    probs = model.predict_proba(vectors)
    for i, x in enumerate(vectors):        
        x_probs = probs[i]
        k_posterior = np.random.choice(components, p=x_probs)
        knock_mean = knock_means_comps_1[k_posterior] + knock_means_comps_2[k_posterior]@x
        knock_cov = knock_covs[k_posterior]
        if i and not i%100:
            print( 'trying',i )
        if not is_pos_semi_def(knock_cov):
            bad_is.append(i)
            continue
        try:
            knockoff_sample = np.random.multivariate_normal(knock_mean, knock_cov)
        except:
            bad_is.append(i)
            continue
        knockoffs.append(knockoff_sample)
        print('success',i)        
    return knockoffs, bad_is 

In [47]:
knockoffs = create_knockoffs(model,np.array(embeddings_spec2vec_lib))

<ipython-input-46-91b430a02389>:40: ComplexWarning: Casting complex values to real discards the imaginary part
  knockoff_sample = np.random.multivariate_normal(knock_mean, knock_cov)


trying 100
trying 200
trying 300
trying 400
trying 500
trying 600
trying 700
trying 800
trying 900
trying 1000
trying 1100
trying 1200
trying 1300
trying 1400
trying 1500
trying 1600
trying 1700
trying 1800
trying 1900
trying 2000
trying 2100
trying 2200
trying 2300
trying 2400
trying 2500
trying 2600
trying 2700
trying 2800
trying 2900
trying 3000
trying 3100
trying 3200
trying 3300
trying 3400
trying 3500
trying 3600
trying 3700
trying 3800
trying 3900
trying 4000
trying 4100
trying 4200
trying 4300
trying 4400
trying 4500
trying 4600
trying 4700
trying 4800
trying 4900
trying 5000
trying 5100
trying 5200
trying 5300
trying 5400
trying 5500
trying 5600
trying 5700
trying 5800
trying 5900
trying 6000
trying 6100
trying 6200
trying 6300
trying 6400
trying 6500
trying 6600
trying 6700
trying 6800
trying 6900
trying 7000
trying 7100
trying 7200
trying 7300
trying 7400
trying 7500
trying 7600
trying 7700
trying 7800
trying 7900
trying 8000
trying 8100
trying 8200
trying 8300
trying 8400
t